# UA Energy News Analytics

_This notebook is used to collect and analyse data from [UA-Energy.org](https://ua-energy.org), a website covering energy news in Ukraine_

***
The analysis starts with **web-scraping** the data from the website and proceed to analysing it using **regular expressions** and data visualisation techniques.

## Table of Contents

[**Web Scraping**](#Web-Scraping)
1. [Obtaining links to articles](#Obtaining-links-to-articles)
2. [Obtaining articles](#Obtaining-articles)
3. [Merging metadata to articles](#Merging-metadata-to-articles)

[**Data Analysis**](#Data-Analysis)
1. [Exploring tags](#Exploring-tags)
2. [Named entity recognition](#Named-entity-recognition)

[**Dashboard**](#Dashboard)

## Libraries

In [1]:
import re

import pandas as pd
import altair as alt

from tqdm import tqdm

import uaenergy as energy

Loaded!


## Web Scraping

<font size = 3>
    In this section, I collect data from the website using a custom-built set of scraping functions. I start by collecting titles, links and dates of articles from a newspage before going on to collect article texts and tags in the next section.
</font>

In [2]:
website = "https://ua-energy.org"
print(website)

https://ua-energy.org


### Obtaining links to articles

<div class="alert alert-warning">

**Note:** The news section of the website is a one-pager, which is not easy to scrape without <a href="https://selenium-python.readthedocs.io">Selenium</a>. However, the news Archive can be used to obtain articles published on a given date. This allows to collect all the articles from the website in an orderly fashion with ease.

</div>

In [3]:
# creating a likely date range for the articles
dates = pd.date_range(start = "2015-01-01", end = "2020-06-13")
dates = [date.strftime('%d-%m-%Y') for date in dates]
print(f"Number of dates: {len(dates)}")

Number of dates: 1991


In [4]:
# scraping titles, dates and links to the articles and putting the data into a dataframe
df_news = pd.concat([energy.parse_news(date) for date in tqdm(dates)], ignore_index=True)
print(f"Shape:{df_news.shape}")
display(df_news.head())

100%|██████████| 1991/1991 [12:01<00:00,  2.76it/s]

Shape:(6605, 3)


,Title,Link,Date
0,"Якщо буде попит з боку Європи чи Китаю, РФ гот...",/uk/posts/yakshcho-bude-popyt-z-boku-yevropy-c...,"23 лютого 2017, 07:11"
1,Насалик у четвер в Кабміні прозвітує про стан ...,/uk/posts/nasalyk-u-chetver-v-kabmini-prozvitu...,"13 березня 2017, 19:38"
2,Україна припинила відбір газу з підземних сховищ,/uk/posts/ukraina-prypynyla-vidbir-hazu-z-pidz...,"13 березня 2017, 13:09"
3,"Будівництво Каховської ГЕС-2 коштуватиме 13,5 ...",/uk/posts/budivnytstvo-kakhovskoi-hes-2-koshtu...,"13 березня 2017, 12:43"
4,Brent дорожчає на тлі бурової активності в США,/uk/posts/brent-dorozhchaie-na-tli-burovoi-akt...,"13 березня 2017, 12:33"


In [5]:
# saving the data
# df_news.to_excel("UAEnergy_Out1_News.xlsx", index = False)
print("Saved!")

Saved!


### Obtaining articles

In [6]:
# reading back the data
df_news = pd.read_excel("UAEnergy_Out1_News.xlsx")
print(f"Shape:{df_news.shape}")
display(df_news.head())

Shape:(6605, 3)


,Title,Link,Date
0,"Якщо буде попит з боку Європи чи Китаю, РФ гот...",/uk/posts/yakshcho-bude-popyt-z-boku-yevropy-c...,"23 лютого 2017, 07:11"
1,Насалик у четвер в Кабміні прозвітує про стан ...,/uk/posts/nasalyk-u-chetver-v-kabmini-prozvitu...,"13 березня 2017, 19:38"
2,Україна припинила відбір газу з підземних сховищ,/uk/posts/ukraina-prypynyla-vidbir-hazu-z-pidz...,"13 березня 2017, 13:09"
3,"Будівництво Каховської ГЕС-2 коштуватиме 13,5 ...",/uk/posts/budivnytstvo-kakhovskoi-hes-2-koshtu...,"13 березня 2017, 12:43"
4,Brent дорожчає на тлі бурової активності в США,/uk/posts/brent-dorozhchaie-na-tli-burovoi-akt...,"13 березня 2017, 12:33"


In [7]:
# obtaining a random link to an article for demonstration
link = df_news["Link"].sample().values[0]
print(link)

/uk/posts/v-ukraini-isnuiut-bilshe-500-vydiv-alternatyvnoho-palyva


In [8]:
# scraping the link with my custom function
energy.get_article_content(link)

('https://ua-energy.org/uk/posts/v-ukraini-isnuiut-bilshe-500-vydiv-alternatyvnoho-palyva',
 'Держенергоефективності опублікувало реєстр альтернативного палива для опалення та пального для авто В Україні зареєстровано 586 видів альтернативного палива, яке можна використовувати для опалення або як автомобільне пальне. Такі дані оприлюднило Держенергоефективності.\xa0 Найбільше зареєстровано видів рідкого палива - більше 300 видів. Частину з них можна використовувати як моторне паливо для авто, частину - як додаток до пального. Твердого палива в Україні теж багато - 284 види. До них відносять паливні брикети, пелети, гранули тощо з різних видів сировини (дерево, тирса, соняшникове лушпиння, солома зернових культур).\xa0 Нагадаємо, в Україні діють стимулюючі тарифи для розвитку альтернативного палива: для станцій, що виробляють електроенергію, діють "зелені" тарифи, а для станцій, що використовують альтернативне паливо і виробляють тепло, встановлено тариф на рівні 90% від газового.\xa0',

In [9]:
# scraping all the links and creating a dataset
df_articles = pd.DataFrame([energy.get_article_content(link) for link in tqdm(df_news["Link"])], 
                           columns = ["Link", "Text", "Tags", "Linked_Articles"])
print(f"Shape:{df_articles.shape}")
display(df_articles.head())

100%|██████████| 6605/6605 [1:44:58<00:00,  1.05it/s]  

Shape:(6605, 4)


,Link,Text,Tags,Linked_Articles
0,https://ua-energy.org/uk/posts/yakshcho-bude-p...,Потенциал действующих газовых месторождений Р...,None,None
1,https://ua-energy.org/uk/posts/nasalyk-u-chetv...,Кабінет міністрів України у четвер заслухає зв...,None,None
2,https://ua-energy.org/uk/posts/ukraina-prypyny...,Україна припинила відбір газу з підземних схов...,None,None
3,https://ua-energy.org/uk/posts/budivnytstvo-ka...,Уряд схвалив техніко-економічне обґрунтування ...,None,None
4,https://ua-energy.org/uk/posts/brent-dorozhcha...,Ціни на нафту знизилися до тримісячного мініму...,None,None


In [10]:
# saving the file in an Excel file
# df_articles.to_excel("UAEnergy_Out2_Articles.xlsx", index = False)
print("Saved!")

Saved!


### Merging metadata to articles

In [11]:
# reading the news file
df_news = pd.read_excel("UAEnergy_Out1_News.xlsx")
print(f"Shape:{df_news.shape}")
display(df_news.head())

Shape:(6605, 3)


,Title,Link,Date
0,"Якщо буде попит з боку Європи чи Китаю, РФ гот...",/uk/posts/yakshcho-bude-popyt-z-boku-yevropy-c...,"23 лютого 2017, 07:11"
1,Насалик у четвер в Кабміні прозвітує про стан ...,/uk/posts/nasalyk-u-chetver-v-kabmini-prozvitu...,"13 березня 2017, 19:38"
2,Україна припинила відбір газу з підземних сховищ,/uk/posts/ukraina-prypynyla-vidbir-hazu-z-pidz...,"13 березня 2017, 13:09"
3,"Будівництво Каховської ГЕС-2 коштуватиме 13,5 ...",/uk/posts/budivnytstvo-kakhovskoi-hes-2-koshtu...,"13 березня 2017, 12:43"
4,Brent дорожчає на тлі бурової активності в США,/uk/posts/brent-dorozhchaie-na-tli-burovoi-akt...,"13 березня 2017, 12:33"


In [12]:
# reading the articles file
df_articles = pd.read_excel("UAEnergy_Out2_Articles.xlsx")
print(f"Shape:{df_articles.shape}")
display(df_articles.head())

Shape:(6605, 4)


,Link,Text,Tags,Linked_Articles
0,https://ua-energy.org/uk/posts/yakshcho-bude-p...,Потенциал действующих газовых месторождений Р...,NaN,NaN
1,https://ua-energy.org/uk/posts/nasalyk-u-chetv...,Кабінет міністрів України у четвер заслухає зв...,NaN,NaN
2,https://ua-energy.org/uk/posts/ukraina-prypyny...,Україна припинила відбір газу з підземних схов...,NaN,NaN
3,https://ua-energy.org/uk/posts/budivnytstvo-ka...,Уряд схвалив техніко-економічне обґрунтування ...,NaN,NaN
4,https://ua-energy.org/uk/posts/brent-dorozhcha...,Ціни на нафту знизилися до тримісячного мініму...,NaN,NaN


In [13]:
# standardasing the links
df_news["Link"] = website + df_news["Link"]

In [14]:
# merging articles with metadata from news
print(f"Shape before:{df_articles.shape}")
df_articles = df_articles.merge(df_news, on = "Link", how = "inner")
print(f"Shape after:{df_articles.shape}")
print("Duplicates count:", df_articles.duplicated().sum())
display(df_articles.head())

Shape before:(6605, 4)
Shape after:(6607, 6)
Duplicates count: 3


,Link,Text,Tags,Linked_Articles,Title,Date
0,https://ua-energy.org/uk/posts/yakshcho-bude-p...,Потенциал действующих газовых месторождений Р...,NaN,NaN,"Якщо буде попит з боку Європи чи Китаю, РФ гот...","23 лютого 2017, 07:11"
1,https://ua-energy.org/uk/posts/nasalyk-u-chetv...,Кабінет міністрів України у четвер заслухає зв...,NaN,NaN,Насалик у четвер в Кабміні прозвітує про стан ...,"13 березня 2017, 19:38"
2,https://ua-energy.org/uk/posts/ukraina-prypyny...,Україна припинила відбір газу з підземних схов...,NaN,NaN,Україна припинила відбір газу з підземних сховищ,"13 березня 2017, 13:09"
3,https://ua-energy.org/uk/posts/budivnytstvo-ka...,Уряд схвалив техніко-економічне обґрунтування ...,NaN,NaN,"Будівництво Каховської ГЕС-2 коштуватиме 13,5 ...","13 березня 2017, 12:43"
4,https://ua-energy.org/uk/posts/brent-dorozhcha...,Ціни на нафту знизилися до тримісячного мініму...,NaN,NaN,Brent дорожчає на тлі бурової активності в США,"13 березня 2017, 12:33"


In [15]:
# exploring duplicates
display(df_articles.loc[df_articles.duplicated(keep = False)])

,Link,Text,Tags,Linked_Articles,Title,Date
2769,https://ua-energy.org/uk/posts/prokuratura-ne-...,Прокуратура не побачила порушень в діях судді ...,NaN,NaN,"Прокуратура не побачила порушень у діях судді,...","05 червня 2018, 12:37"
2770,https://ua-energy.org/uk/posts/prokuratura-ne-...,Прокуратура не побачила порушень в діях судді ...,NaN,NaN,"Прокуратура не побачила порушень у діях судді,...","05 червня 2018, 12:37"
2771,https://ua-energy.org/uk/posts/prokuratura-ne-...,Прокуратура не побачила порушень в діях судді ...,NaN,NaN,"Прокуратура не побачила порушень у діях судді,...","05 червня 2018, 12:37"
2772,https://ua-energy.org/uk/posts/prokuratura-ne-...,Прокуратура не побачила порушень в діях судді ...,NaN,NaN,"Прокуратура не побачила порушень у діях судді,...","05 червня 2018, 12:37"


In [16]:
# arranging the columns in the desired order
to_order = ['Link', 'Title', 'Date', 'Text', 'Tags', 'Linked_Articles']

In [17]:
# dropping duplicated rows and reordering the columns
print(f"Shape before:{df_articles.shape}")
df_articles.drop_duplicates(inplace = True)
df_articles = df_articles.reindex(to_order, axis = 1)
print(f"Shape after:{df_articles.shape}")
display(df_articles.head())

Shape before:(6607, 6)
Shape after:(6604, 6)


,Link,Title,Date,Text,Tags,Linked_Articles
0,https://ua-energy.org/uk/posts/yakshcho-bude-p...,"Якщо буде попит з боку Європи чи Китаю, РФ гот...","23 лютого 2017, 07:11",Потенциал действующих газовых месторождений Р...,NaN,NaN
1,https://ua-energy.org/uk/posts/nasalyk-u-chetv...,Насалик у четвер в Кабміні прозвітує про стан ...,"13 березня 2017, 19:38",Кабінет міністрів України у четвер заслухає зв...,NaN,NaN
2,https://ua-energy.org/uk/posts/ukraina-prypyny...,Україна припинила відбір газу з підземних сховищ,"13 березня 2017, 13:09",Україна припинила відбір газу з підземних схов...,NaN,NaN
3,https://ua-energy.org/uk/posts/budivnytstvo-ka...,"Будівництво Каховської ГЕС-2 коштуватиме 13,5 ...","13 березня 2017, 12:43",Уряд схвалив техніко-економічне обґрунтування ...,NaN,NaN
4,https://ua-energy.org/uk/posts/brent-dorozhcha...,Brent дорожчає на тлі бурової активності в США,"13 березня 2017, 12:33",Ціни на нафту знизилися до тримісячного мініму...,NaN,NaN


In [18]:
# saving the final dataset to an Excel file
df_articles.to_excel("UAEnergy_Out3_MergedArticles.xlsx", index = False)
print("Saved!")

Saved!


## Data Analysis

<font size = 3>
    In this section, I perform exploratory data analysis to answer the following questions:
    <ol>
        <li>how frequently are articles published?</li>
        <li>which tags are most common on the website?</li>
        <li>how often are some key organisations and countries mentioned in texts?</li>
        <li>who are the most frequently mentioned persons on the wesite?</li>
    </ol>
</font>

In [2]:
# reading the full dataset
df_articles = pd.read_excel("UAEnergy_Out3_MergedArticles.xlsx")
print(f"Shape:{df_articles.shape}")
display(df_articles.head())

Shape:(6604, 6)


,Link,Title,Date,Text,Tags,Linked_Articles
0,https://ua-energy.org/uk/posts/yakshcho-bude-p...,"Якщо буде попит з боку Європи чи Китаю, РФ гот...","23 лютого 2017, 07:11",Потенциал действующих газовых месторождений Р...,NaN,NaN
1,https://ua-energy.org/uk/posts/nasalyk-u-chetv...,Насалик у четвер в Кабміні прозвітує про стан ...,"13 березня 2017, 19:38",Кабінет міністрів України у четвер заслухає зв...,NaN,NaN
2,https://ua-energy.org/uk/posts/ukraina-prypyny...,Україна припинила відбір газу з підземних сховищ,"13 березня 2017, 13:09",Україна припинила відбір газу з підземних схов...,NaN,NaN
3,https://ua-energy.org/uk/posts/budivnytstvo-ka...,"Будівництво Каховської ГЕС-2 коштуватиме 13,5 ...","13 березня 2017, 12:43",Уряд схвалив техніко-економічне обґрунтування ...,NaN,NaN
4,https://ua-energy.org/uk/posts/brent-dorozhcha...,Brent дорожчає на тлі бурової активності в США,"13 березня 2017, 12:33",Ціни на нафту знизилися до тримісячного мініму...,NaN,NaN


In [3]:
# converting strings to objects and date string to datetime
df_articles["Tags"] = df_articles["Tags"].fillna('1').apply(eval).replace({1:None})
df_articles["Linked_Articles"] = df_articles["Linked_Articles"].fillna('1').apply(eval).replace({1:None})
df_articles["Date"] = df_articles["Date"].apply(energy.replace_months)

In [4]:
# some descriptives of the dataset
df_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6604 entries, 0 to 6603
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Link             6604 non-null   object        
 1   Title            6604 non-null   object        
 2   Date             6604 non-null   datetime64[ns]
 3   Text             6603 non-null   object        
 4   Tags             5192 non-null   object        
 5   Linked_Articles  1553 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 309.7+ KB


In [5]:
# there are some articles with no tags and many articles with no links to other articles
display(df_articles.isna().sum())

Link                  0
Title                 0
Date                  0
Text                  1
Tags               1412
Linked_Articles    5051
dtype: int64

In [6]:
# removing one link with no text
print(f"Shape before:{df_articles.shape}")
df_articles.dropna(subset=["Text"], inplace = True)
print(f"Shape after:{df_articles.shape}")

Shape before:(6604, 6)
Shape after:(6603, 6)


In [7]:
# calculating the number of articles per month
df_temp = df_articles.set_index("Date").groupby(pd.Grouper(freq = "M")).agg(Count = ("Link", "count"))
df_temp.reset_index(inplace = True)
print(f"Shape:{df_temp.shape}")
display(df_temp.head())

Shape:(41, 2)


,Date,Count
0,2017-02-28,1
1,2017-03-31,6
2,2017-04-30,49
3,2017-05-31,134
4,2017-06-30,145


In [8]:
chart1 = alt.Chart(df_temp).mark_line(point=True, size =2.5, strokeDash=[1,2]).encode(
    x=alt.X("Date"),
    y="Count",
    tooltip=["Date", "Count"]
).properties(
    width = 750,
    # height = 450,
    title = {"text":"Figure 1. Monthly Count of Articles on UA-Energy.org",
             "subtitle":f"Counts Based on Regex Matches"}

).interactive()
chart1

alt.Chart(...)

In [9]:
# keeping only articles after April 1, 2017
print(f"Shape before:{df_articles.shape}")
df_articles = df_articles[df_articles['Date'].ge("2017-04-01")]
print(f"Shape after:{df_articles.shape}")
display(df_articles.head())

Shape before:(6603, 6)
Shape after:(6596, 6)


,Link,Title,Date,Text,Tags,Linked_Articles
7,https://ua-energy.org/uk/posts/u-kharkivskii-o...,У Харківській області виявили велике газове ро...,2017-04-12 12:10:00,У Харківській області знайдено родовище газу і...,{'/uk/tags/ukrhazvydoubvannia': 'Укргазвидоубв...,None
8,https://ua-energy.org/uk/posts/hazprom-pidpysa...,"""Газпром"" підписав із словацькою Eustream на т...",2017-04-12 11:52:00,"""Газпром експорт"" уклав зі словацькою Eustream...","{'/uk/tags/hazprom': 'Газпром', '/uk/tags/haz'...",None
9,https://ua-energy.org/uk/posts/hazprom-ne-zmih...,"""Газпром"" не зміг відновити судовий розгляд з ...",2017-04-13 15:18:00,«Газпрому» не удалось возобновить судебное раз...,"{'/uk/tags/hazprom': 'Газпром', '/uk/tags/lytv...",None
10,https://ua-energy.org/uk/posts/sud-vidpustyv-d...,Суд відпустив другого фігуранта Сергія Перелом...,2017-04-24 17:45:00,Солом'янський районний суд Києва відпустив пер...,"{'/uk/tags/pereloma': 'Перелома', '/uk/tags/ma...",None
11,https://ua-energy.org/uk/posts/nabu-hotuie-spr...,НАБУ готує справу щодо “Енергоатому” для перед...,2017-04-24 17:42:00,Національне антикорупційне бюро готує справу щ...,"{'/uk/tags/enerhoatom': 'Енергоатом', '/uk/tag...",None


### Exploring tags

#### Top 10 tags 

In [10]:
# delecting articles with tags
df_temp = df_articles[["Link", "Tags"]].dropna()
print(f"Shape:{df_temp.shape}")

# extracting tags
df_temp["Tags"] = df_temp["Tags"].apply(lambda x: list(x.values()))
df_temp = df_temp.explode("Tags")
print(f"Shape after:{df_temp.shape}")

display(df_temp.head())

Shape:(5190, 2)
Shape after:(14465, 2)


,Link,Tags
7,https://ua-energy.org/uk/posts/u-kharkivskii-o...,Укргазвидоубвання
7,https://ua-energy.org/uk/posts/u-kharkivskii-o...,газ
8,https://ua-energy.org/uk/posts/hazprom-pidpysa...,Газпром
8,https://ua-energy.org/uk/posts/hazprom-pidpysa...,газ
8,https://ua-energy.org/uk/posts/hazprom-pidpysa...,Eustream


In [11]:
# there are about 1.4k unique tags in total
df_temp.nunique()

Link    5190
Tags    1410
dtype: int64

In [12]:
print(f"Shape before:{df_temp.shape}")

# aggregating by tag and calculating the number of articles with a given tag
df_temp = df_temp.groupby("Tags", as_index = False).count()
df_temp.rename({"Tags":"Tag", "Link":"Frequency"}, axis = 1, inplace = True)
df_temp.sort_values("Frequency", ascending = False, inplace = True)

print(f"Shape after:{df_temp.shape}")
display(df_temp.head())

Shape before:(14465, 2)
Shape after:(1410, 2)


,Tag,Frequency
851,газ,816
475,Нафтогаз,704
465,НКРЕКП,326
1121,нафта,280
1345,транзит,267


In [13]:
# saving top 5 tags for future use
top_tags = df_temp["Tag"].head().tolist()

In [14]:
chart2 = alt.Chart(df_temp.head(10)).mark_bar().encode(
    x=alt.X("Frequency", title = "Number of Articles"),
    y=alt.Y("Tag", sort = "-x"), #-x for descending order
    tooltip=["Tag", "Frequency"]
).properties(
    height = 250,
    width = 250,
    title = {"text":"Figure 2. Top 10 Tags by the Number of Articles ",
             "subtitle":f"Based on {df_articles.Tags.dropna().shape[0]} Articles with Tags"}

).interactive()
chart2

alt.Chart(...)

#### Top 5 tags in time

In [15]:
# delecting articles with tags
df_temp = df_articles[["Link", "Tags", "Date"]].dropna()
print(f"Shape:{df_temp.shape}")

# extracting tags
df_temp["Tags"] = df_temp["Tags"].apply(lambda x: list(x.values()))
df_temp = df_temp.explode("Tags")
print(f"Shape after:{df_temp.shape}")

# keeping only observations for top 5 links and setting Date to be the index
df_temp = df_temp.query("Tags in @top_tags")
df_temp.set_index('Date', inplace = True)
print(f"Shape after:{df_temp.shape}")

display(df_temp.head())

Shape:(5190, 3)
Shape after:(14465, 3)
Shape after:(2393, 2)


,Link,Tags
Date,,
2017-04-12 12:10:00,https://ua-energy.org/uk/posts/u-kharkivskii-o...,газ
2017-04-12 11:52:00,https://ua-energy.org/uk/posts/hazprom-pidpysa...,газ
2017-04-24 17:19:00,https://ua-energy.org/uk/posts/nahliadovu-radu...,Нафтогаз
2017-04-25 14:41:00,https://ua-energy.org/uk/posts/rozmovy-pro-des...,газ
2017-04-25 13:55:00,https://ua-energy.org/uk/posts/chystyi-zbytok-...,газ


In [16]:
# calculating the total number of articles in a given quarter
df_n = df_temp.groupby([pd.Grouper(freq = "Q")]).agg(N_Articles = ('Link','nunique'))
print(f"Shape:{df_n.shape}")
display(df_n.head())

Shape:(13, 1)


,N_Articles
Date,
2017-06-30,72
2017-09-30,86
2017-12-31,57
2018-03-31,145
2018-06-30,118


In [17]:
print(f"Shape before:{df_temp.shape}")

# aggregating by quarter and calculating the number of articles with a given tags in a given quarter
df_temp = df_temp.groupby([pd.Grouper(freq = "Q"), "Tags"]).count()

# adding the total number of articles per quarter
df_temp = df_temp.join(df_n)

# calculating the share of articles with a given tag in a given quarter
df_temp.eval("Share = Link / N_Articles * 100", inplace = True)
df_temp['Share'] = df_temp['Share'].round(1)

# prettifying the dataframe
df_temp.reset_index(inplace = True)
df_temp.rename({"Tags":"Tag"}, axis = 1, inplace = True)

print(f"Shape after:{df_temp.shape}")
display(df_temp.head())

Shape before:(2393, 2)
Shape after:(65, 5)


,Date,Tag,Link,N_Articles,Share
0,2017-06-30,НКРЕКП,5,72,6.9
1,2017-06-30,Нафтогаз,18,72,25.0
2,2017-06-30,газ,33,72,45.8
3,2017-06-30,нафта,18,72,25.0
4,2017-06-30,транзит,2,72,2.8


In [18]:
selection = alt.selection_multi(fields=['Tag'], bind='legend')

chart3 = alt.Chart(df_temp).mark_line(point=True, size =2.5, strokeDash=[1,2]).encode(
    x=alt.X("Date", axis=alt.Axis(format = ("%b %Y"), labelAngle=90)),
    y=alt.Y("Share", title="Share of Articles"),
    color = "Tag",
    tooltip=["Tag", "Date", "Share"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width = 400,
    height = 250,
    title = {"text":"Figure 3. Quarterly Share of Articles Mentioning Top 5 Tags",
             "subtitle":f"Based on {df_articles.Tags.dropna().shape[0]} Articles with Tags"}

).add_selection(
    selection
)
chart3

alt.Chart(...)

### Named entity recognition

In [19]:
# obtaining a random article for demonstration
text = df_articles['Text'].sample().values[0]
print(text)

Заступник міністра прокоментував можливість розвитку державної вугільної галузі в Україні
 З державних вугільних шахт перспективи розвитку мають 10 шахт, а більше 20 шахт — збиткові. Про це розповів заступник міністра енергетики та захисту довкілля Віталій Шубін, повідомляє "Українська енергетика". “Понад 10 шахт мають перспективи розвитку і видобування гарного і потрібного вугілля. І вас запевняю, ці шахти будуть працювати. Зараз ми шукаємо формулу, яким чином зробити галузь беззбитковою”, — сказав Шубін.  Він відзначив, що міністерство почало аудит державних шахт, більше 20 з яких — збиткові.  “Я думаю, що українцям вже набридло постійно фінансувати вугільну галузь, оскільки вона на себе не заробляє. Це вже факт — вона не заробляє на останні 10 років. Дуже дивно, що приватні шахти на себе заробляють, а державні — не заробляють”, — сказав Шубін. Нагадаємо, уряд пропонує Верховній Раді переглянути державний бюджет на 2019 рік та перенаправити  1 млрд грн на виплату заробітних плат шахт

#### Top 5 abbreviations

In [20]:
# extracting block letters which are often organisations, countries or other relevant entities
pattern = "[А-ЯІЇЄ]{2,}"
print(re.findall(pattern, text)[:20])

[]


In [21]:
# extracting abbreviations from all texts and presenting 30 most frequent ones
df_articles['Abbr'] = df_articles['Text'].str.findall(pattern)
df_articles['Abbr'].explode().value_counts().head(30)

НАК       2307
ПАТ       1660
НКРЕКП    1559
АТ        1296
ЄС        1249
ГТС       1041
МВ        1031
США       1016
ДТЕК      1001
ТОВ        927
ТЕС        829
АЕС        793
ДП         735
ТЕЦ        613
ПСГ        507
ВДЕ        494
НАЕК       455
РФ         448
НЕК        447
НАБУ       388
ПДВ        376
АМКУ       340
ОСББ       292
ОПЕК       276
НПЗ        270
АЗС        250
ЄБРР       249
СЕС        243
МВФ        237
ПСО        215
Name: Abbr, dtype: int64

In [22]:
# abbreviations of interest to be looked up in the text
lookup = ["ЄС", "МВФ", "ЄЄБРР", "США", "РФ"]

In [23]:
# delecting columns of interest
df_temp = df_articles[["Link", "Date", "Abbr"]].copy()
print(f"Shape:{df_temp.shape}")
display(df_temp.head())

Shape:(6596, 3)


,Link,Date,Abbr
7,https://ua-energy.org/uk/posts/u-kharkivskii-o...,2017-04-12 12:10:00,[ПАТ]
8,https://ua-energy.org/uk/posts/hazprom-pidpysa...,2017-04-12 11:52:00,[]
9,https://ua-energy.org/uk/posts/hazprom-ne-zmih...,2017-04-13 15:18:00,"[СК, СК, СК, ПАО, ПАО]"
10,https://ua-energy.org/uk/posts/sud-vidpustyv-d...,2017-04-24 17:45:00,"[НАК, НАК, НАБУ, САП, ПАО, ОГХК, САП, ИФ, СИЗО..."
11,https://ua-energy.org/uk/posts/nabu-hotuie-spr...,2017-04-24 17:42:00,"[НАБУ, НАБУ, УНН, НАБУ, ДП, НАЕК]"


In [24]:
print(f"Shape before:{df_temp.shape}")

# unlisting column values
df_temp = df_temp.explode("Abbr")
print(f"Shape after:{df_temp.shape}")

# keeping only rows where a relevant abbreviation is mentioned
df_temp = df_temp.query("Abbr in @lookup")
df_temp.set_index("Date", inplace = True)
print(f"Shape after:{df_temp.shape}")

display(df_temp.head())

Shape before:(6596, 3)
Shape after:(32371, 3)
Shape after:(2950, 2)


,Link,Abbr
Date,,
2017-04-24 17:31:00,https://ua-energy.org/uk/posts/u-rf-kazhut-shc...,РФ
2017-04-24 17:31:00,https://ua-energy.org/uk/posts/u-rf-kazhut-shc...,РФ
2017-04-24 17:31:00,https://ua-energy.org/uk/posts/u-rf-kazhut-shc...,РФ
2017-04-24 17:31:00,https://ua-energy.org/uk/posts/u-rf-kazhut-shc...,РФ
2017-04-26 17:09:00,https://ua-energy.org/uk/posts/sap-podala-apel...,США


In [25]:
# calculating the total number of articles in a given month
df_n = df_articles.set_index('Date').groupby([pd.Grouper(freq = "M")]).agg(N_Articles = ('Link','nunique'))
print(f"Shape:{df_n.shape}")
display(df_n.head())

Shape:(39, 1)


,N_Articles
Date,
2017-04-30,49
2017-05-31,134
2017-06-30,145
2017-07-31,257
2017-08-31,166


In [26]:
print(f"Shape before:{df_temp.shape}")

# aggregating by month and abbreviation to calculate the number of articles mentioning a given entity
df_temp = df_temp.groupby([pd.Grouper(freq = "M"), "Abbr"]).agg({"Link":"nunique"})

# adding the total number of articles per quarter
df_temp = df_temp.join(df_n)

# calculating the share of articles mentioni a given tag in a given quarter
df_temp.eval("Share = Link / N_Articles * 100", inplace = True)
df_temp['Share'] = df_temp['Share'].round(1)

# prettifying the dataframe
df_temp.reset_index(inplace = True)
df_temp.rename({"Abbr":"Entity", "Link":"Frequency"}, axis = 1, inplace = True)

print(f"Shape after:{df_temp.shape}")
display(df_temp.head())

Shape before:(2950, 2)
Shape after:(148, 5)


,Date,Entity,Frequency,N_Articles,Share
0,2017-04-30,ЄС,2,49,4.1
1,2017-04-30,МВФ,1,49,2.0
2,2017-04-30,РФ,2,49,4.1
3,2017-04-30,США,4,49,8.2
4,2017-05-31,ЄС,8,134,6.0


In [27]:
selection = alt.selection_multi(fields=['Entity'], bind='legend')

chart4 = alt.Chart(df_temp).mark_line(point=True, size =2.5, strokeDash=[1,2]).encode(
    x=alt.X("Date", axis=alt.Axis(format = ("%b %Y"))),
    y=alt.Y("Share", title="Share of Articles"),
    color = "Entity",
    tooltip=["Entity", "Date", "Share"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width = 750,
    # height = 450,
    title = {"text":"Figure 4. Monthly Share of Articles Mentioning 5 Selected Entities",
             "subtitle":f"Entity Mentions are Based on Regex Matches"}

).add_selection(
    selection
)
chart4

alt.Chart(...)

#### Top persons

In [28]:
# Extracting full proper names
pattern = "[А-ЯІЇЄ][а-яіїє]{2,} [А-ЯІЇЄ][а-яіїє]{2,}"
print(re.findall(pattern, text)[:20])

['Віталій Шубін', 'Верховній Раді']


In [29]:
# Dmanually excluding some flase positives
to_exclude = ["Украї", "Верховн", "Російсь"]

In [30]:
# extracting names from all texts
df_articles["Names"] = df_articles["Text"].str.findall(pattern)

In [31]:
# some most commonly mentioned names
for name in df_articles["Names"].explode().value_counts().index[:10]:
    if all([w not in name for w in to_exclude]):
        print(name)

Андрій Коболєв
Олексій Оржель
Володимир Гройсман
Юрій Вітренко
Ольга Буславець
Олексій Гончарук


In [32]:
# using surnames to claculate mentions
lookup = ["Коболєв", "Оржель", "Гройсман", "Вітренко", "Буславець", "Гончарук"]

In [33]:
# selecting relevant columns
df_temp = df_articles[["Link", "Date", "Text"]].copy()

# determining if the person is mentioned in the text or not
for name in lookup:
    df_temp[name] = df_temp["Text"].str.contains(name)

df_temp.set_index('Date', inplace = True)
print(f"Shape:{df_temp.shape}")

display(df_temp.head())

Shape:(6596, 8)


,Link,Text,Коболєв,Оржель,Гройсман,Вітренко,Буславець,Гончарук
Date,,,,,,,,
2017-04-12 12:10:00,https://ua-energy.org/uk/posts/u-kharkivskii-o...,У Харківській області знайдено родовище газу і...,False,False,False,False,False,False
2017-04-12 11:52:00,https://ua-energy.org/uk/posts/hazprom-pidpysa...,"""Газпром експорт"" уклав зі словацькою Eustream...",False,False,False,False,False,False
2017-04-13 15:18:00,https://ua-energy.org/uk/posts/hazprom-ne-zmih...,«Газпрому» не удалось возобновить судебное раз...,False,False,False,False,False,False
2017-04-24 17:45:00,https://ua-energy.org/uk/posts/sud-vidpustyv-d...,Солом'янський районний суд Києва відпустив пер...,False,False,False,False,False,False
2017-04-24 17:42:00,https://ua-energy.org/uk/posts/nabu-hotuie-spr...,Національне антикорупційне бюро готує справу щ...,False,False,False,False,False,False


In [34]:
print(f"Shape before:{df_temp.shape}")

# aggregating data by month and calculating the number of articles mentioning each person
df_temp = df_temp.groupby(pd.Grouper(freq = "M"))[lookup].sum()

# calculating the share of articles
df_temp = df_temp.divide(df_n["N_Articles"], axis = 0).multiply(100).round(1)

# prettifying the dataframe
df_temp.reset_index(inplace = True)
print(f"Shape after:{df_temp.shape}")

# converting from wide to long format
df_temp = df_temp.melt(id_vars = "Date", var_name = "Name", value_name = "Share")
print(f"Shape after:{df_temp.shape}")

# selecting only observations with 1 or more mentions
df_temp = df_temp.query("Share > 0")
print(f"Shape after:{df_temp.shape}")

display(df_temp.head())

Shape before:(6596, 8)
Shape after:(39, 7)
Shape after:(234, 3)
Shape after:(145, 3)


,Date,Name,Share
0,2017-04-30,Коболєв,2.0
1,2017-05-31,Коболєв,1.5
2,2017-06-30,Коболєв,6.2
3,2017-07-31,Коболєв,1.6
4,2017-08-31,Коболєв,2.4


In [35]:
selection = alt.selection_multi(fields=['Name'], bind='legend')

chart5 = alt.Chart(df_temp).mark_line(point=True, size =2.5, strokeDash=[1,2]).encode(
    x=alt.X("Date", axis =alt.Axis(format = ("%b %Y"))),
    y=alt.Y("Share", title="Share of Articles"),
    color = "Name",
    tooltip=["Name", "Date", "Share"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width = 750,
    title = {"text":"Figure 5. Monthly Share of Articles Mentioning Top 6 Persons",
             "subtitle":f"Person Mentions are Based on Regex Matches"}

).add_selection(
    selection
)
chart5

alt.Chart(...)

## Dashboard

<div class="alert alert-info">

**Note:** The below dashboard is interactive. For example, you can scale the figures, see additionlal information in tooltips or highlight specific lines using legends.

</div>

In [36]:
alt.vconcat(
    chart1, alt.hconcat(chart2, chart3), chart4, chart5,
    title = alt.TitleParams("Dashboard on Text Data from UA-Energy.org",
                            fontSize=24, anchor='start', offset = 30)
).resolve_scale(
    color='independent'
).configure_point(
    size=70
)

alt.VConcatChart(...)